<a href="https://colab.research.google.com/github/ImVis10/1PercentStrategy/blob/V1/1PercentStrategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv  
import re
import datetime

In [2]:
API_KEY = 'OTPBXZ24V7Q9A24S'

In [3]:
def write_to_csv(stock,data, mode):


  # header = ['time', 'open', 'high', 'low', 'close', 'volume']

  with open(stock+'_minute.csv', mode, encoding='UTF8') as f:
      writer = csv.writer(f)

      # write the header
      # writer.writerow(header)

      # write the data
      writer.writerow(data)

In [ ]:
import csv
import requests

def get_stock_data(stock_name):
  time_periods ={
    # '1':'year1month1',
    # '2':'year1month2',
    # '3':'year1month3',
    # '4':'year1month4',
    # '5':'year1month5',
    # '6':'year1month6',
    # '7':'year1month7',
    # '8':'year1month8',
    # '9':'year1month9',
    # '10':'year1month10',
    # '11':'year1month11',
    # '12':'year1month12',
    # '13':'year2month1',
    # '14':'year2month2',
    # '15':'year2month3',
    # '16':'year2month4',
    # '17':'year2month5',
    # '18':'year2month6',
    # '19':'year2month7',
    # '20':'year2month8',
    # '21':'year2month9',
    # '22':'year2month10',
    # '23':'year2month11',
    # '24':'year2month12',
}
  for month, time_period in time_periods.items():
    CSV_URL = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=%(stock)s&interval=1min&slice=%(slice)s&apikey=%(key)s&adjusted=false'\
            %{'stock':stock_name,'slice':time_period,'key':API_KEY}
    with requests.Session() as s:
      download = s.get(CSV_URL)
      decoded_content = download.content.decode('utf-8')
      cr = csv.reader(decoded_content.splitlines(), delimiter=',')
      my_list = list(cr)
      for row in my_list:
          write_to_csv('test', row,'a')
get_stock_data('IBM')          

In [203]:
stock = 'AAPL'
mode = 'r'
interval = 'minute'
file_path = stock+'_' + interval +'.csv'
AAPL_minute_data = []
with open(file_path) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
            AAPL_minute_data.append(list(row))
print('done')
print(len(AAPL_minute_data))

done
406974


In [216]:
daily_opening_candle = []


In [217]:
date_and_time = AAPL_minute_data[0][0].split()
current_date = date_and_time[0]
for i in range(len(AAPL_minute_data)):
  date_and_time = AAPL_minute_data[i][0].split()
  end_date = date_and_time[0]
  

  # if there is a change in the current_date, get the data
  if (current_date != end_date):
    daily_opening_candle.append(AAPL_minute_data[i-1])
    current_date = end_date

daily_opening_candle.append(AAPL_minute_data[-1])
daily_opening_candle.reverse()

In [218]:
daily_closing_candle = []

In [219]:
current_date = ''
for i in range(len(AAPL_minute_data)):
  date_and_time = AAPL_minute_data[i][0].split()
  end_date = date_and_time[0]

  # if there is a change in the current_date, get the data
  if (current_date != end_date):
    daily_closing_candle.append(AAPL_minute_data[i])
    current_date = end_date

daily_closing_candle.reverse()  


In [220]:
# Initialize 
PRINCIPAL = 1000
returns = []
num_trades = 0

In [221]:
def close_trade(gain_or_loss):
  global PRINCIPAL
  global num_trades
  returns.append(gain_or_loss)
  PRINCIPAL += gain_or_loss
  num_trades+=1


In [222]:
# Principal
# Current_principal = Principal - buying_price
# Once stock_price meets desired profit(gain)
# Exit trade => Current_principal = current_principal + buying_price + Gain
########################### Go bull ##################

# Buy
# Wait until you make desired profit or hit the SL
# Exit
def go_bull(profit_percent,buy_price,time_series_data):
  target = buy_price + (buy_price * profit_percent)
  
  stop_loss = profit_percent/2
  bottom_line = buy_price - (buy_price*stop_loss)
  # Start from second entry because we entered at first entry
  for row in time_series_data[1:]:
    # Check for stop loss first
    #  Comparing with open and low 
    if(max(float(row[1])  , float(row[3]))  <= bottom_line):
      close_trade(max(float(row[1])  , float(row[3])) - buy_price)
      return None
    # Comparing with open and high 
    if(float(row[1]) , float(row[2])  >= target):
      close_trade(max(float(row[1])  , float(row[2])) - buy_price)
      return None
  close_trade(buy_price - float(time_series_data[-1][4]))   


In [223]:
# Principal
# Current_principal = Principal - selling_price
# Once stock_price meets desired profit(gain)
# Exit trade => Current_principal = current_principal + selling_price + Gain
########################### Go bear ##################

# Sell
# Wait until you make desired profit or hit the SL
# Exit

def go_bear(profit_percent, sell_price, time_series_data):
  target = sell_price - (sell_price * profit_percent)
  stop_loss = profit_percent/2
  top_line = sell_price + (sell_price*stop_loss)


# Start from second entry because we entered at first entry
  for row in time_series_data[1:]:
    # Check for stop loss first
    #  Comparing with open and high
    if(min(float(row[1]), float(row[2]))  >= top_line):
      close_trade(sell_price - min(float(row[1])  , float(row[2])))
      return None
    # Comparing with open and low 
    if(min(float(row[1]),float( row[3]))  <= target):
      close_trade( sell_price - min(float(row[1])  , float(row[3])))
      return None
  # Neither conditions hit. Close at the end of the trading day 
  close_trade(sell_price - float(time_series_data[-1][4]))   

In [224]:
def get_minute_data(date):

  minute_data = []

  # Column header should not be considered hence len - 1
  for i in range(1, len(AAPL_minute_data) - 1):
      current_date = AAPL_minute_data[i][0].split()[0]
      if (current_date == date):
        minute_data.append(AAPL_minute_data[i])
        next_date = AAPL_minute_data[i+1][0].split()[0]
        if (current_date != next_date):
          break
  minute_data.reverse()
  return minute_data  


In [225]:
def run_simulation(profit_percentage,diff_between_trading_sessions):
  for i in range(len(daily_opening_candle)-1):
    
    prev_stock_price = float(daily_closing_candle[i][4])
    stock_price = float(daily_opening_candle[i+1][4])

    bullish_condition = stock_price > prev_stock_price * (1 + diff_between_trading_sessions)
    bearish_condition = stock_price < prev_stock_price * (1 - diff_between_trading_sessions)

    if(bullish_condition):
      time_series = get_minute_data(daily_opening_candle[i+1][0].split()[0])
      go_bull(profit_percentage, stock_price,time_series)

    elif(bearish_condition):
      time_series = get_minute_data(daily_opening_candle[i+1][0].split()[0])
      go_bear(profit_percentage, stock_price,time_series)




In [ ]:
PRINCIPAL = 1000
num_trades = 0
profit_percentage = 1/100
diff_between_trading_sessions = 0.5/100

run_simulation(profit_percentage, diff_between_trading_sessions)

In [228]:
print(PRINCIPAL)
print(num_trades)
print(returns)

1007.597600000001
163
[-0.18999999999999773, 0.06999999999999318, 0.11000000000001364, 0.09000000000000341, 0.0, -0.7199999999999989, -0.03999999999999204, 0.6400000000000148, -0.7599999999999909, 0.06999999999999318, 0.30000000000001137, 0.01999999999998181, -0.020000000000010232, -1.9599999999999937, 0.09999999999999432, -0.730000000000004, 0.3100000000000023, -0.0799999999999983, -0.6299999999999955, 0.46000000000000796, 0.20000000000000284, -0.6000000000000085, 1.3299999999999983, 0.0, 0.0, -0.09000000000000341, 0.09000000000000341, -0.6400000000000006, 0.0, 0.21000000000000796, 0.07999999999998408, 1.4399999999999977, 0.18000000000000682, 1.2750000000000057, 0.3100000000000023, -0.6199999999999903, -0.0799999999999983, 0.0, -0.6800000000000068, 1.460000000000008, 0.0, -0.8000000000000114, 0.21999999999999886, -0.18000000000000682, -0.08999999999997499, 1.5400000000000205, -0.8799999999999955, 0.030000000000001137, 1.4899999999999807, 0.1599999999999966, 0.0, -0.75, 1.4900000000000